This script calculates the centroids and vertices of objects detected using YOLO's Oriented Bounding Box (OBB) format, and saves the results into a CSV file.

**Note:** Coordinate Reference Systems (CRS) with metric units (e.g., UTM) are more accurate than GPS coordinates (latitude/longitude) for calculating distances between objects.

In [1]:
import os
import json
import pandas as pd
from shapely.geometry import Polygon
from tqdm import tqdm
from rasterio import open as rio_open
from rasterio.transform import xy
from pyproj import CRS, Transformer

# Configuration
TILE_SIZE = 1024
json_folder = "/shared/data/climateplus2025/Prediction_for_poster_July21/Predicion/predicted_json"
tile_folder = "/shared/data/climateplus2025/Prediction_for_poster_July21/CapeTown_Image_2023_tiles_1024_for_prediction"
output_csv = "processed_centroids_and_bbox.csv"

# CRS
custom_crs_wkt = """
PROJCS["Hartebeesthoek94_Lo19_(E-N)",
    GEOGCS["Hartebeesthoek94",
        DATUM["Hartebeesthoek94",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6148"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4148"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",19],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["ESRI","102562"]]
"""
source_crs = CRS.from_wkt(custom_crs_wkt)
target_crs = CRS.from_epsg(4326)  # WGS84 GPS
transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)

# Functions
def parse_tile_name(image_name):
    parts = image_name.split("_tile_")[-1].split(".")[0]
    x_offset, y_offset = map(int, parts.split("_"))
    return x_offset, y_offset

def get_pixel_centroid(polygon):
    poly = Polygon(polygon)
    return poly.centroid.x, poly.centroid.y

def get_bounding_box_corners(polygon):
    poly = Polygon(polygon)
    minx, miny, maxx, maxy = poly.bounds
    return [
        (minx, miny),  # bottom-left
        (maxx, miny),  # bottom-right
        (maxx, maxy),  # top-right
        (minx, maxy)   # top-left
    ]

def extract_year(image_name):
    for part in image_name.split("_"):
        if part.isdigit() and len(part) == 4:
            return int(part)
    return None

def pixel_to_crs_coords(tile_path, pixel_x, pixel_y):
    with rio_open(tile_path) as src:
        transform = src.transform
        crs_x, crs_y = xy(transform, pixel_y, pixel_x, offset="center")
        return crs_x, crs_y

def crs_to_gps(crs_xy):
    try:
        x, y = crs_xy
        lon, lat = transformer.transform(x, y)
        return round(lon, 8), round(lat, 8)
    except Exception:
        return None, None

# Processing
results = []
json_files = sorted([f for f in os.listdir(json_folder) if f.endswith(".json")])

for file in tqdm(json_files, desc="Processing JSONs"):
    try:
        with open(os.path.join(json_folder, file), "r") as f:
            data = json.load(f)

        img_name = data["image_name"]
        tile_path = os.path.join(tile_folder, img_name)

        for pred in data.get("predictions", []):
            polygon = pred["polygon"]
            cx, cy = get_pixel_centroid(polygon)
            crs_cx, crs_cy = pixel_to_crs_coords(tile_path, cx, cy)
            lon, lat = crs_to_gps((crs_cx, crs_cy))

            bbox_pixels = get_bounding_box_corners(polygon)
            bbox_crs = [pixel_to_crs_coords(tile_path, x, y) for x, y in bbox_pixels]
            bbox_gps = [crs_to_gps(p) for p in bbox_crs]

            result = {
                "prediction_id": pred["prediction_id"],
                "image_name": img_name,
                "year": extract_year(img_name),
                "class": pred["class"],
                "confidence": pred["confidence"],
                "pixel_centroid": (round(cx, 2), round(cy, 2)),
                "crs_centroid": (round(crs_cx, 2), round(crs_cy, 2)),
                "gps_centroid": (lon, lat)
            }

            for i, (pix, crs, gps) in enumerate(zip(bbox_pixels, bbox_crs, bbox_gps), start=1):
                result[f"bbox_pixel_{i}"] = tuple(round(v, 2) for v in pix)
                result[f"bbox_crs_{i}"] = tuple(round(v, 2) for v in crs)
                result[f"bbox_gps_{i}"] = gps

            results.append(result)

    except Exception as e:
        print(f"Skipping {file}: {e}")

# Save DataFrame
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)
print(f"Saved: {output_csv}")
print(f"Total: {len(df)} | Valid GPS centroids: {df['gps_centroid'].notna().sum()}")

# Sample Output
print("\n[Sample Output (centroid + bbox → CRS → GPS)]")
for i in range(min(3, len(df))):
    row = df.iloc[i]
    print(f"Centroid: {row['pixel_centroid']} → {row['crs_centroid']} → {row['gps_centroid']}")
    for j in range(1, 5):
        print(f"  bbox{j}: pixel {row[f'bbox_pixel_{j}']} → CRS {row[f'bbox_crs_{j}']} → GPS {row[f'bbox_gps_{j}']}")


Processing JSONs: 100%|██████████| 207/207 [00:22<00:00,  9.40it/s]

Saved: processed_centroids_and_bbox.csv
Total: 810 | Valid GPS centroids: 810

[Sample Output (centroid + bbox → CRS → GPS)]
Centroid: (869.25, 791.57) → (-28930.42, -3743882.57) → (18.68750215, -33.82128875)
  bbox1: pixel (846.0, 773.0) → CRS (-28932.28, -3743881.08) → GPS (18.68748211, -33.82127531)
  bbox2: pixel (892.0, 773.0) → CRS (-28928.6, -3743881.08) → GPS (18.68752185, -33.82127541)
  bbox3: pixel (892.0, 810.0) → CRS (-28928.6, -3743884.04) → GPS (18.68752176, -33.82130209)
  bbox4: pixel (846.0, 810.0) → CRS (-28932.28, -3743884.04) → GPS (18.68748201, -33.82130199)
Centroid: (703.25, 45.96) → (-28943.7, -3743822.92) → (18.68736066, -33.82075062)
  bbox1: pixel (673.0, 30.0) → CRS (-28946.12, -3743821.64) → GPS (18.68733456, -33.82073905)
  bbox2: pixel (734.0, 30.0) → CRS (-28941.24, -3743821.64) → GPS (18.68738727, -33.82073918)
  bbox3: pixel (734.0, 62.0) → CRS (-28941.24, -3743824.2) → GPS (18.68738719, -33.82076226)
  bbox4: pixel (673.0, 62.0) → CRS (-28946.12, -37